In [2]:
!pip install tensorflow-gpu

  Using cached numpy-1.19.5-cp36-cp36m-manylinux2010_x86_64.whl (14.8 MB)
  Using cached numpy-1.19.5-cp36-cp36m-manylinux2010_x86_64.whl (14.8 MB)


  Attempting uninstall: numpy
    Found existing installation: numpy 1.16.1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.16.1
    Uninstalling numpy-1.16.1:
      Successfully uninstalled numpy-1.16.1
    Uninstalling numpy-1.16.1:
      Successfully uninstalled numpy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-cpu 2.1.3 requires gast==0.2.2, but you have gast 0.3.3 which is incompatible.
tensorflow-cpu 2.1.3 requires keras-preprocessing==1.1.0, but you have keras-preprocessing 1.1.2 which is incompatible.
tensorflow-cpu 2.1.3 requires numpy<1.19.0,>=1.16.0, but you have numpy 1.19.5 which is incompatible.
tensorflow-cpu 2.1.3 requires tensorboard<2.2.0,>=2.1.0, but you have tensorboard 2.5.0 which is incompatible.
tensorflow-cpu 2.1.3 requires tensorflow-estimator<2.2.0,>=2.1.0rc0, but you have tensorflow-estimat

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Conv3D, ZeroPadding3D
from tensorflow.keras.layers import MaxPooling3D
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Bidirectional, TimeDistributed
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Lambda
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam

In [4]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    y_pred = y_pred[:, :, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

def CTC(name, args):
    return Lambda(ctc_lambda_func, output_shape=(1,), name=name)(args)

In [5]:
class LipNet(object):
    def __init__(self, img_c=3, img_w=100, img_h=50, frames_n=75, absolute_max_string_len=32, output_size=28):
        self.img_c = img_c
        self.img_w = img_w
        self.img_h = img_h
        self.frames_n = frames_n
        self.absolute_max_string_len = absolute_max_string_len
        self.output_size = output_size
        self.build()

    def build(self):
        if K.image_data_format() == 'channels_first':
            input_shape = (self.img_c, self.frames_n, self.img_w, self.img_h)
        else:
            input_shape = (self.frames_n, self.img_w, self.img_h, self.img_c)

        self.input_data = Input(name='the_input', shape=input_shape, dtype='float32')

        self.zero1 = ZeroPadding3D(padding=(1, 2, 2), name='zero1')(self.input_data)
        self.conv1 = Conv3D(32, (3, 5, 5), strides=(1, 2, 2), activation='relu', kernel_initializer='he_normal', name='conv1')(self.zero1)
        self.maxp1 = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max1')(self.conv1)
        self.drop1 = Dropout(0.5)(self.maxp1)

        self.zero2 = ZeroPadding3D(padding=(1, 2, 2), name='zero2')(self.drop1)
        self.conv2 = Conv3D(64, (3, 5, 5), strides=(1, 1, 1), activation='relu', kernel_initializer='he_normal', name='conv2')(self.zero2)
        self.maxp2 = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max2')(self.conv2)
        self.drop2 = Dropout(0.5)(self.maxp2)

        self.zero3 = ZeroPadding3D(padding=(1, 1, 1), name='zero3')(self.drop2)
        self.conv3 = Conv3D(96, (3, 3, 3), strides=(1, 1, 1), activation='relu', kernel_initializer='he_normal', name='conv3')(self.zero3)
        self.maxp3 = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max3')(self.conv3)
        self.drop3 = Dropout(0.5)(self.maxp3)

        self.resh1 = TimeDistributed(Flatten())(self.drop3)

        self.gru_1 = Bidirectional(GRU(256, return_sequences=True, kernel_initializer='Orthogonal', name='gru1'), merge_mode='concat')(self.resh1)
        self.gru_2 = Bidirectional(GRU(256, return_sequences=True, kernel_initializer='Orthogonal', name='gru2'), merge_mode='concat')(self.gru_1)

        # transforms RNN output to character activations:
        self.dense1 = Dense(self.output_size, kernel_initializer='he_normal', name='dense1')(self.gru_2)

        self.y_pred = Activation('softmax', name='softmax')(self.dense1)

        self.labels = Input(name='the_labels', shape=[self.absolute_max_string_len], dtype='float32')
        self.input_length = Input(name='input_length', shape=[1], dtype='int64')
        self.label_length = Input(name='label_length', shape=[1], dtype='int64')

        self.loss_out = CTC('ctc', [self.y_pred, self.labels, self.input_length, self.label_length])

        self.model = Model(inputs=[self.input_data, self.labels, self.input_length, self.label_length], outputs=self.loss_out)
    
    def summary(self):
        Model(inputs=self.input_data, outputs=self.y_pred).summary()

In [6]:
lipnet = LipNet(3,50,100,150,150,32)

In [7]:
adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
lipnet.model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=adam)

In [8]:
import pandas as pd
!pip install numpy==1.16.1
import numpy as np

  Using cached numpy-1.16.1-cp36-cp36m-manylinux1_x86_64.whl (17.3 MB)
  Using cached numpy-1.16.1-cp36-cp36m-manylinux1_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-gpu 2.4.1 requires numpy~=1.19.2, but you have numpy 1.16.1 which is incompatible.
tensorflow-cpu 2.1.3 requires gast==0.2.2, but you have gast 0.3.3 which is incompatible.
tensorflow-cpu 2.1.3 requires keras-preprocessing==1.1.0, but you have keras-preprocessing 1.1.2 which is incompatible.
tensorflow-cpu 2.1.3 requires tensorboard<2.2.0,>=2.1.0, but you have tensorb

In [9]:
x = pd.read_csv('x_input.csv')
y = pd.read_csv('y_label.csv')
x = x.drop('Unnamed: 0',axis=1)
y = y.drop('Unnamed: 0',axis=1)
y['ctc_text'] += '____'
dataset = pd.merge(x, y)
dataset=dataset.sample(frac=1).reset_index(drop=True)

In [10]:
x_train = dataset.loc[:0.7*len(dataset)-1]
x_val = dataset.loc[0.7*len(dataset):0.9*len(dataset)-1]
x_test = dataset.loc[0.9*len(dataset):]
x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)
assert len(dataset) == len(x_train) + len(x_val) + len(x_test)

In [11]:
tokens = {"_": 0, "<s>": 1, "</s>": 2, "<unk>": 3, "|": 4, "E": 5, "T": 6, "A": 7, "O": 8, "N": 9, "I": 10, "H": 11, "S": 12, "R": 13, "D": 14, "L": 15, "U": 16, "M": 17, "W": 18, "C": 19, "F": 20, "G": 21, "Y": 22, "P": 23, "B": 24, "V": 25, "K": 26, "'": 27, "X": 28, "J": 29, "Q": 30, "Z": 31}
rev_tokens = {value : key for (key, value) in tokens.items()}
extra_tokens = { 'bos_token' :"<s>" , 'eos_token' :"</s>" , 'unk_token' : "<unk>" , 'pad_token' : "_" , 'word_delimiter_token' : "|" }
rev_extra_tokens = {value : key for (key, value) in extra_tokens.items()}

In [12]:
def generator(csv, start, batch_size = 16):
    inputs = []
    labels = []
    input_length = np.ones((batch_size,1))*150
    label_length = np.ones((batch_size,1))*150
    for i in range(min(batch_size,len(csv)-start)):
        x = np.load(csv['output_paths'][start+i])
        #x = np.transpose(x , (0,2,1,3))
        y = []
        for c in csv['ctc_text'][start+i]:
            y.append(tokens[c])
        y = np.array(y)
        inputs.append(x)
        labels.append(y)
        
    return [np.array(inputs),np.array(labels),input_length,label_length] , np.zeros((batch_size,))

In [13]:
epochs = 1
start_train = 0
batch_size_train = 1
start_val = 0
batch_size_val = 1

In [14]:
for i in range(23100):
    
    xt,yt = generator(x_train, start_train, batch_size = batch_size_train)
    start_train += batch_size_train
    if start_train >= len(x_train):
        start_train = 0

    #print(xt[0].shape,xt[1].shape,xt[2].shape,xt[3].shape)

    '''xv,yv = generator(x_val, start_val, batch_size = batch_size_val)
    start_val += batch_size_val
    if start_val >= len(x_val):
        start_val = 0'''

    print('Epochs : ',epochs)
    print('Start : ',start_train)
    try:
        h = lipnet.model.fit(xt,yt,epochs = 1)
        epochs += 1
    except:
        print('Error')
        continue

Epochs :  1
Start :  1
Epochs :  1
Start :  1
1/1 [==============================] - 8s 8s/step - loss: inf
Epochs :  2
Start :  2
Epochs :  2
Start :  2
1/1 [==============================] - 2s 2s/step - loss: inf
Epochs :  3
Start :  3
Epochs :  3
Start :  3
1/1 [==============================] - 2s 2s/step - loss: inf
Epochs :  4
Start :  4
Epochs :  4
Start :  4
1/1 [==============================] - 2s 2s/step - loss: inf
Epochs :  5
Start :  5
Epochs :  5
Start :  5
1/1 [==============================] - 2s 2s/step - loss: inf
Epochs :  6
Start :  6
Epochs :  6
Start :  6
1/1 [==============================] - 2s 2s/step - loss: inf
Epochs :  7
Start :  7
Epochs :  7
Start :  7
1/1 [==============================] - 2s 2s/step - loss: inf
Epochs :  8
Start :  8
Epochs :  8
Start :  8
Error
Error
Epochs :  8
Start :  9
Epochs :  8
Start :  9
Error
Error
Epochs :  8
Start :  10
Epochs :  8
Start :  10
Error
Error
Epochs :  8
Start :  11
Epochs :  8
Start :  11
1/1 [===============

ValueError: Object arrays cannot be loaded when allow_pickle=False

ValueError: Object arrays cannot be loaded when allow_pickle=False